In [ ]:
import json
import pandas as pd
import ollama  # Ensure you have installed this package via pip

In [15]:
# Example DataFrame
df = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'tweet': [
        "I fucking hate starbucks coffee, so expensive",
        "There is no starbucks near my city",
        "I love starbucks caramel malchiato",
        "I dont like the new printed notes on the coffee..."
    ],
    'PRODUCT': [0, 0, 0, 0],
    'PLACE': [0, 0, 0, 0],
    'PRICE': [0, 0, 0, 0]
})

def classify_tweet(tweet):
    # Define a prompt with clear instructions and desired JSON output
    prompt = f"""
You are an expert NLP classifier. Given the tweet below, please determine which categories it belongs to among PRODUCT, PLACE, and PRICE.
Output the answer in JSON format exactly as:
{{"PRODUCT": 0 or 1, "PLACE": 0 or 1, "PRICE": 0 or 1}}

Tweet: "{tweet}"
Only output the JSON.
"""
    # Instantiate the Ollama client correctly (without model parameter in constructor)
    client = ollama.Client()
    
    # Run the model with the prompt, specify model and temperature here
    result_str = client.chat(
        model="llama3",
        messages=[{"role": "user", "content": prompt}],
        options={"temperature": 0}
    )
    
    # Extract the content from the response
    response_content = result_str['message']['content'].strip()
    
    # Parse and return the JSON result
    try:
        return json.loads(response_content)
    except json.JSONDecodeError:
        # Fallback in case the model doesn't return valid JSON
        print(f"Error parsing JSON: {response_content}")
        return {"PRODUCT": 0, "PLACE": 0, "PRICE": 0}

In [16]:
# Apply the classification function to each tweet
results = []
for tweet in df['tweet']:
    try:
        results.append(classify_tweet(tweet))
    except Exception as e:
        print(f"Error processing tweet: {e}")
        results.append({"PRODUCT": 0, "PLACE": 0, "PRICE": 0})

# Convert results to a DataFrame and update the original DataFrame
for i, result in enumerate(results):
    df.at[i, 'PRODUCT'] = result.get('PRODUCT', 0)
    df.at[i, 'PLACE'] = result.get('PLACE', 0)
    df.at[i, 'PRICE'] = result.get('PRICE', 0)

print(df)

   id                                              tweet  PRODUCT  PLACE  \
0   1      I fucking hate starbucks coffee, so expensive        1      1   
1   2                 There is no starbucks near my city        1      1   
2   3                 I love starbucks caramel malchiato        1      1   
3   4  I dont like the new printed notes on the coffe...        1      1   

   PRICE  
0      1  
1      0  
2      0  
3      0  


In [17]:
print(results)

[{'PRODUCT': 1, 'PLACE': 1, 'PRICE': 1}, {'PRODUCT': 1, 'PLACE': 1, 'PRICE': 0}, {'PRODUCT': 1, 'PLACE': 1, 'PRICE': 0}, {'PRODUCT': 1, 'PLACE': 1, 'PRICE': 0}]
